# Get results for combined model

In [162]:
import os
import numpy as np
import pandas as pd

from CoMPARA_experiment_helper import load_data, get_model, data_scaling, file_updater 
from CoMPARA_experiment_helper import get_model_setup_params, get_model_compile_params#, setup_callback_paths
# from AOTexperiment_helper import cont_model_eval, get_cont_model_score
from CoMPARA_experiment_helper import get_model_predictions

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.keras.backend.set_floatx('float32')


data_dict = load_data()
train_features = data_dict['train_features']
test_features = data_dict['test_features']
train_targets = data_dict['train_targets']
test_targets = data_dict['test_targets']
train_Fweights = data_dict['Fweights_train']
test_Fweights = data_dict['Fweights_test']
labels = data_dict['labels']

In [163]:
#Get scaled data
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
target_scaler = None
binary_labels = [0, 2, 4]
train_features_scaled, test_features_scaled, train_targets_scaled, test_targets_scaled, feature_scaler, target_scaler = data_scaling(
    feature_scaler,
    target_scaler,
    train_features,
    test_features,
    train_targets,
    test_targets
)
n_feat = train_features_scaled.shape[1]

def get_valid_ind(y):
    ind = np.where(~np.isnan(y.astype(np.float32)))[0]
    return ind

In [164]:
#Update this for each model_type
# model_type = "dense"
# super_folder = os.path.join("CoMPARA_singlelabelmodels",
#                             "final_210429_dense_lr0_005")

# model_type = "LS"
# super_folder = os.path.join("CoMPARA_singlelabelmodels",
#                             "final_210429_LS_lr0_005")

# model_type = "LSwFW"
# super_folder = os.path.join("CoMPARA_singlelabelmodels",
#                             "final_210520_LSwFW_lr0_001")

# model_type = "LSwFW_ones"
# super_folder = os.path.join("CoMPARA_singlelabelmodels",
#                             "final_210520_LSwFW_lr0_001")

# f = os.walk(super_folder)
# folder_names = next(f)[1]



model_type = "xgboost"
super_folder = "CoMPARA_singlelabelmodels"
folder_names = [
    "210528_xgboost_AgonistClass", 
    "210528_xgboost_AntagonistClass",
    "210528_xgboost_BindingClass"
]
output_prefix = "_".join([model_type, "CoMPARA"])
column_headers = labels[binary_labels]


In [166]:
# Get predictions for each model_type
train_predicts = []
test_predicts = []
for idx, label_idx in enumerate(binary_labels):
    valid_train_ind = get_valid_ind(train_targets_scaled.values[:, label_idx])
    valid_test_ind = get_valid_ind(test_targets_scaled.values[:, label_idx])
    
    endpoint="binary"
    n_endpoints=None
#     n_endpoints = None
#     if label_idx < 2:
#         endpoint = "binary"
#     elif label_idx == 2:
#         endpoint = "regression"
#     else:
#         endpoint = "multiclass"
#         n_endpoints = len(np.unique(
#             train_targets_scaled[valid_train_ind, label_idx]))

    checkpoint_path = os.path.join(super_folder,
                                   folder_names[idx],
                                   "model_checkpoint"
                                   )
    X_train = train_features_scaled
    X_test = test_features_scaled
    if model_type=="xgboost":
        import xgboost as xgb
        checkpoint_path = os.path.join(super_folder, 
                                       folder_names[idx]
                                      )
    elif model_type=="LSwFW":
        X_train = np.hstack([train_features_scaled,train_Fweights])
        X_test = np.hstack([test_features_scaled, test_Fweights])
    elif model_type =="LSwFW_ones":
        X_train = np.hstack([train_features_scaled, 
                             np.ones_like(train_features_scaled)])
        X_test = np.hstack([test_features_scaled,
                            np.ones_like(test_features_scaled)
                           ])
    #Get predictions for all datapoints
    a, b = get_model_predictions(
        model_type,
        checkpoint_path,
        endpoint=endpoint,
        X_train=X_train,
        X_test=X_test,
        n_feat=n_feat,
        target_scaler=target_scaler,
        n_endpoints=n_endpoints
    )
    if a.ndim==1:
        a = a.reshape(-1,1)
        b = b.reshape(-1,1)
    train_predicts.append(a)
    test_predicts.append(b)

train_predicts = pd.DataFrame(np.hstack(train_predicts),
                              columns=column_headers)
test_predicts = pd.DataFrame(np.hstack(test_predicts),
                             columns=column_headers)
train_predicts.to_csv("_".join([output_prefix,
                                "train_predict"
                                ]), index=False)
test_predicts.to_csv("_".join([output_prefix,
                               "test_predict"
                               ]), index=False)

## Get combined predictions

In [18]:
labels

Index(['AgonistClass', 'Potency_Class_Agonist', 'AntagonistClass',
       'Potency_Class_Antagonist', 'BindingClass', 'Potency_Class_Binding'],
      dtype='object')

In [42]:
# folder_names = [
#     "210517_dense_AOT_wFP_very_toxic_",
#     "210517_LS_AOT_wFP_very_toxic_",
#     "210517_LSwFW_AOT_wFP_very_toxic_",
#     "210517_LSwFW_ones_AOT_wFP_very_toxic_"
# ]

label = "BindingClass"
label_idx = 4

folder_names = [
    os.path.join("final_210429_dense_lr0_005",
        f"210429_Dense_lr0_005_CoMPARA_binary_{label}"),
    os.path.join("final_210429_LS_lr0_005",
                 f"210429_LS_lr0_005_CoMPARA_binary_{label}"),
    os.path.join("final_210520_LSwFW_lr0_001",
        f"210520_LSwFW_lr0_001_CoMPARA_binary_{label}"),
    os.path.join("final_210520_LSwFW_ones_lr0_001",
        f"210520_LSwFW_ones_lr0_001_CoMPARA_binary_{label}")
]

# folder_names = [
#     "210515_dense_AOT_regression_LD50_mgkg",
#     "210515_LS_AOT_regression_LD50_mgkg",
#     "210516_LSwFW_AOT_regression_LD50_mgkg",
#     "210516_LSwFW_ones_AOT_regression_LD50_mgkg"
# ]

model_types = ["dense", "LS", "LSwFW", "LSwFW_ones"]

In [43]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
# Get predictions for each model_type
train_predicts = []
test_predicts = []
for idx, model_type in enumerate(model_types):
    valid_train_ind = get_valid_ind(
        train_targets_scaled.values[:, label_idx])
    valid_test_ind = get_valid_ind(
        test_targets_scaled.values[:, label_idx])
    endpoint = "binary"
    n_endpoints = None

    checkpoint_path = os.path.join(
        "CoMPARA_singlelabelmodels",
        folder_names[idx],
        "model_checkpoint"
    )
    if model_type == "LSwFW":
        X_train = np.hstack([train_features_scaled[valid_train_ind],
                             train_Fweights[valid_train_ind]
                             ])
        X_test = np.hstack([test_features_scaled[valid_test_ind],
                            test_Fweights[valid_test_ind]
                            ])
    elif model_type == "LSwFW_ones":
        X_train = np.hstack([
            train_features_scaled[valid_train_ind],
            np.ones_like(train_Fweights[valid_train_ind])
        ])
        X_test = np.hstack([
            test_features_scaled[valid_test_ind],
            np.ones_like(test_Fweights[valid_test_ind])
        ])
    else:
        X_train = train_features_scaled[valid_train_ind]
        X_test = test_features_scaled[valid_test_ind]
    a, b = get_model_predictions(
        model_type,
        checkpoint_path,
        endpoint=endpoint,
        X_train=X_train,
        X_test=X_test,
        n_feat=n_feat,
        target_scaler=target_scaler,
        n_endpoints=n_endpoints
    )
#     if endpoint =="multiclass":
#         a = np.expand_dims(np.argmax(a, axis=1)+1, axis=1)
#         b = np.expand_dims(np.argmax(b, axis=1)+1, axis=1)
    train_predicts.append(a)
    test_predicts.append(b)

train_predicts = pd.DataFrame(np.hstack(train_predicts),
                              )  # columns=model_types)
test_predicts = pd.DataFrame(np.hstack(test_predicts),
                             )  # columns=model_types)
# train_predicts.to_csv("_".join([output_prefix,
#                                 "train_predict"
#                                 ]), index=False)
# test_predicts.to_csv("_".join([output_prefix,
#                                "test_predict"
#                                ]), index=False)

First dense layer with 1200 hidden unites


In [44]:
from CoMPARA_experiment_helper import get_cm, get_sn_sp, get_qual_model_score
def get_model_eval(y_train, y_test, train_predict, test_predict, num_classes=2):
    if num_classes>2:
        sn_train_list, sp_train_list = [],[]
        sn_test_list, sp_test_list=[],[]
        for i in range(num_classes):
            if len(np.unique(y_train[:, i]))<2:
                continue
            cm = get_cm(y_train[:,i], train_predict[:,i], )
            sn_train, sp_train = get_sn_sp(cm)
            cm = get_cm(y_test[:,i], test_predict[:,i], )
            sn_test, sp_test = get_sn_sp(cm)
            sn_train_list.append(sn_train)
            sp_train_list.append(sp_train)
            sn_test_list.append(sn_test)
            sp_test_list.append(sp_test)
        sn_train = np.mean(sn_train_list)
        sp_train = np.mean(sp_train_list)
        sn_test = np.mean(sn_test_list)
        sp_test = np.mean(sp_test_list)
    else:
        cm = get_cm(y_train, train_predict, num_classes=num_classes)
        sn_train, sp_train = get_sn_sp(cm)
        cm = get_cm(y_test, test_predict)
        sn_test, sp_test = get_sn_sp(cm)
    model_score = get_qual_model_score(sn_train, sp_train, sn_test, sp_test)
    return model_score, (sn_train, sp_train, sn_test, sp_test)
    

In [52]:
model_ind=[0,1]

y_train = train_targets_scaled.values[valid_train_ind, label_idx]
y_test = test_targets_scaled.values[valid_test_ind, label_idx]
combined_train_predict = np.mean(
    train_predicts.values[:,model_ind], axis=1)
combined_test_predict = np.mean(test_predicts.values[:,model_ind],
                       axis=1)
if endpoint=="binary":
    r=get_model_eval(y_train.astype(int), y_test.astype(int), 
               train_predict=combined_train_predict>0.5,
               test_predict=combined_test_predict>0.5,
              )
    print(r)
elif endpoint=="multiclass":
    combined_train_predict = [train_predicts.values[:, 
            i*n_endpoints:(i+1)*n_endpoints]
                             for i in model_ind]
    combined_train_predict = to_categorical(np.argmax(
        np.mean(combined_train_predict, axis=0),
        axis=1
    ), num_classes=n_endpoints)
    combined_test_predict = [test_predicts.values[:, 
            i*n_endpoints:(i+1)*n_endpoints] 
                             for i in model_ind]
    combined_test_predict = to_categorical(np.argmax(
        np.mean(combined_test_predict, axis=0), 
        axis=1
    ), num_classes=n_endpoints)
    y_train = to_categorical(y_train.astype(int)-1)
    y_test = to_categorical(y_test.astype(int)-1)
    r=get_model_eval(y_train, y_test, 
                     train_predict=combined_train_predict,
                     test_predict=combined_test_predict,
                     num_classes=n_endpoints
                    )    
    print(r)
else:
    y_train = np.log10(y_train.astype(np.float32))
    y_test = y_test.astype(np.float32)
    ind = np.where(y_test>10000.)[0]
    y_test[ind]=10000.
    y_test = np.log10(y_test)
    
    from sklearn.metrics import r2_score, mean_squared_error
    r2_train = r2_score(y_train, combined_train_predict)
    r2_test = r2_score(y_test, combined_test_predict)
    model_score = (0.3*r2_train) + (0.45*r2_test) +(0.25*(1-np.abs(r2_train-r2_test)))
    print(model_score, r2_train, r2_test)


(0.8107716141643768, (0.9595959595959596, 0.950136612021858, 0.577433628318584, 0.9067055393586005))


# Train RF/XGBoost "Stacking" model

In [53]:
import numpy as np
import pandas as pd
import os
import time

In [101]:
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import roc_auc_score
def tune_xgbc(params, X_train, y_train, X_test, y_test, regression=False, 
	objective = "reg:logistic", eval_metric='logloss'):
# Implementation learned on a lesson of Mario Filho (Kagle Grandmaster) for parametes optmization.
# Link to the video: https://www.youtube.com/watch?v=WhnkeasZNHI

	"""Function to be passed as scikit-optimize minimizer/maximizer input

	Parameters:
	Tuples with information about the range that the optimizer should use for that parameter, 
	as well as the behaviour that it should follow in that range.

	Returns:
	float: the metric that should be minimized. If the objective is maximization, then the negative 
	of the desired metric must be returned. In this case, the negative AUC average generated by CV is returned.
	"""


	#Hyperparameters to be optimized
	print(params)
	learning_rate = params[0] 
	n_estimators = params[1] 
	max_depth = params[2]
	min_child_weight = params[3]
	gamma = params[4]
	subsample = params[5]
	colsample_bytree = params[6]


	#Model to be optimized
	if regression:
		sample_weight = None  	
	
		mdl = xgb.XGBRegressor(learning_rate = learning_rate, 
							n_estimators = n_estimators, 
							max_depth = max_depth, 
							min_child_weight = min_child_weight, 
							gamma = gamma, 
							subsample = subsample, 
							colsample_bytree = colsample_bytree, 
							objective=objective,
							use_label_encoder=False,
							seed = 42
							)
	else:
		sample_weight = compute_sample_weight("balanced", y_train)    	
		mdl = xgb.XGBClassifier(learning_rate = learning_rate, 
							n_estimators = n_estimators, 
							max_depth = max_depth, 
							min_child_weight = min_child_weight, 
							gamma = gamma, 
							subsample = subsample, 
							colsample_bytree = colsample_bytree, 
							objective=objective,
							use_label_encoder=False,
							eval_metric=eval_metric,
							seed = 42)



	# #Cross-Validation in order to avoid overfitting
	# auc = cross_val_score(mdl, X_train_selected, y_train, cv = 10, scoring = 'roc_auc')

	mdl.fit(X_train, y_train, sample_weight = sample_weight)
	if regression:
		test_predict = mdl.predict(X_test)
		res = mean_squared_error(y_test, test_predict, squared=False)
	else:
		test_predict = mdl.predict_proba(X_test)
		y_test=y_test.astype(int)
		if test_predict.shape[1]==2:
			test_predict=test_predict[:, 1]
		res = roc_auc_score(y_test, test_predict, multi_class = 'ovr')

	print(res.mean())
	# as the function is minimization (forest_minimize), we need to use the negative of the desired metric (AUC)
	return -res.mean()


In [102]:
from CoMPARA_experiment_helper import load_data, get_model, data_scaling, file_updater 

data_dict = load_data()
train_features = data_dict['train_features']
test_features = data_dict['test_features']
train_targets = data_dict['train_targets']
test_targets = data_dict['test_targets']
train_Fweights = data_dict['Fweights_train']
test_Fweights = data_dict['Fweights_test']
labels = data_dict['labels']

#Get scaled data
from sklearn.preprocessing import StandardScaler
feature_scaler = StandardScaler()
target_scaler = None
binary_labels = [0, 2, 4]
train_features_scaled, test_features_scaled, train_targets_scaled, test_targets_scaled, feature_scaler, target_scaler = data_scaling(
    feature_scaler,
    target_scaler,
    train_features,
    test_features,
    train_targets,
    test_targets
)
n_feat = train_features_scaled.shape[1]

def get_valid_ind(y):
    ind = np.where(~np.isnan(y.astype(np.float32)))[0]
    return ind

In [157]:
import xgboost as xgb
label_idx = binary_labels[2]

valid_train_ind = get_valid_ind(
    train_targets.values[:, label_idx])
valid_test_ind = get_valid_ind(
    test_targets.values[:,label_idx])
y_train = train_targets.values[valid_train_ind, label_idx].astype(int)
y_test = test_targets.values[valid_test_ind, label_idx].astype(int)
X_train = train_features_scaled[valid_train_ind]
X_test = test_features_scaled[valid_test_ind]


In [158]:
from functools import partial
from skopt import forest_minimize
import dill as pickle
# from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
# sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=42)
# for _train_ind, _val_ind in sss.split(list(range(len(y_train))), y_train):
#     break

space = [(1e-3, 1e-1, 'log-uniform'), # learning rate
        (100, 2000), # n_estimators
        (1, 10), # max_depth 
        (1, 6.), # min_child_weight 
        (0, 0.5), # gamma 
        (0.5, 1.), # subsample 
        (0.5, 1.)] # colsample_bytree 
func = partial(tune_xgbc, 
               X_train = X_train,
               y_train = y_train, 
               X_test = X_test,
               y_test = y_test, 
               objective = "binary:logistic",
               regression = False,
               eval_metric="logloss"
              )
result = forest_minimize(func, space, random_state=42, n_random_starts=20, n_calls=25, verbose=1)

with open(f"xgboost_{labels[label_idx]}_params.ob", 'wb') as f:
    pickle.dump(result, f)

Iteration No: 1 started. Evaluating function at random point.
[0.03918194347141743, 1394, 8, 3.9932924209851834, 0.07800932022121827, 0.5779972601681014, 0.5290418060840998]
0.8388461389612736
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 10.0760
Function value obtained: -0.8388
Current minimum: -0.8388
Iteration No: 2 started. Evaluating function at random point.
[0.05399484409787437, 1223, 8, 4.540362888980228, 0.010292247147901225, 0.9849549260809973, 0.916221320400211]
0.8383085218916897
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 12.8790
Function value obtained: -0.8383
Current minimum: -0.8388
Iteration No: 3 started. Evaluating function at random point.
[0.0026587543983272693, 1315, 5, 4.087407548138583, 0.3058265802441405, 0.5035331526098588, 0.5115312125207079]
0.8385552387832503
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 13.1573
Function value obtained: -0.8386
Current minimum: -0.8388
Iteration No: 4 star

In [159]:
with open(f"xgboost_{labels[label_idx]}_params.ob", 'rb') as f:
    params = pickle.load(f)
params = params['x']
learning_rate = params[0] 
n_estimators = params[1] 
max_depth = params[2]
min_child_weight = params[3]
gamma = params[4]
subsample = params[5]
colsample_bytree = params[6]

In [160]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight("balanced", y_train)
model = xgb.XGBClassifier(
                          learning_rate=learning_rate,
                          n_estimators=n_estimators,
                          max_depth=max_depth,
                          min_child_weight=min_child_weight,
                          gamma=gamma,
                          subsample=subsample,
                          colsample_bytree=colsample_bytree,
                          use_label_encoder=False,
                          objective="binary:logistic",
                          eval_metric="logloss",
                          random_state=42)
model.fit(X_train,
          y_train,
          sample_weight=sample_weights
          )
model.save_model(os.path.join("CoMPARA_singlelabelmodels", 
                              "_".join([time.strftime("%y%m%d", time.localtime()), 
                                                                     "xgboost", labels[label_idx] 
                                       ] )))

c:\users\cavio\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:444: UserWarning: n_estimators is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.')
c:\users\cavio\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:444: UserWarning: max_depth is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.')


In [161]:
train_predict = model.predict(
    train_features_scaled[valid_train_ind])
test_predict = model.predict(
    test_features_scaled[valid_test_ind])
r=get_model_eval(
    y_train,
    y_test, 
    train_predict=train_predict>0.5,
    test_predict=test_predict>0.5,
    num_classes=2
)
print(r)

(0.7139177416858019, (1.0, 1.0, 0.3185840707964602, 0.9892128279883382))
